### Reading data

In [ ]:
import pandas as pd
import numpy as np
import ast

def parse_combined_column(s):
    if s == '':
        return np.zeros(768)
    else:
        return np.array(eval(s))
    

df = pd.read_csv('final_data_with_features_2.csv', converters={
    'bert': parse_combined_column,
    'auto': parse_combined_column,
    'roberta': parse_combined_column
})

### Filter out non-relevant dates

In [ ]:
# Function to filter rows based on first non-NaN headline
def filter_after_first_non_nan(group):
    first_non_nan_index = group['headline'].first_valid_index()
    if first_non_nan_index is not None:
        return group.loc[first_non_nan_index:]
    else:
        return pd.DataFrame(columns=group.columns)

# Apply the function to each group
filtered_df = df.groupby('stock').apply(filter_after_first_non_nan).reset_index(drop=True)

print(filtered_df)

### Making final data

In [ ]:
def change_sentiment_label(x):
    if x == 'POSITIVE':
        return 1
    elif x == 'NEGATIVE':
        return -1
    else:
        return 0

def change_sentiment_score(x):
    if -1<=x<=1 :
        return x
    else:
        return 0
    
filtered_df['sentiment_label'] = filtered_df['sentiment_label'].apply(change_sentiment_label)
filtered_df['sentiment_score'] = filtered_df['sentiment_score'].apply(change_sentiment_score)
# set sentiment score to be the sentiment score times the sentiment label
filtered_df['sentiment_score'] = filtered_df['sentiment_score'] * filtered_df['sentiment_label']

In [ ]:
import pandas as pd

# Assuming `filtered_df` is your DataFrame
half_point = len(filtered_df) // 2

# Split the DataFrame into two parts
df_part1 = filtered_df.iloc[:half_point]
df_part2 = filtered_df.iloc[half_point:]

# Save each part to a separate CSV file
df_part1.to_csv('filtered_df_part1.csv', index=False)
df_part2.to_csv('filtered_df_part2.csv', index=False)